# Splitting articles published in more than one issue

This scripts help to clean up pages and issue statements in bibliographic records of Die Gartenlaube especially for articles published splitted up over serveral issues.

[GitHub Issue](https://github.com/DieDatenlaube/DieDatenlaube/issues/4)

In [24]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import re

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT * WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?Die_Gartenlaube wdt:P1433 wd:Q655617; 
                   wdt:P304 ?pages; 
                   wdt:P433 ?issue
  FILTER(REGEX(?issue,";|–"))
  FILTER(REGEX(?pages, ",|;"))
}
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent="#DieDatenlaube")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result["issue"]["value"])
    print(result["pages"]["value"])
    #issues = result["issue"]["value"].split("–|;") #means a range
    issues = re.split("–|–|;|,",result["issue"]["value"])

    #Überprüfen ob einzelne issues ebenfalls bereits Heftnr.reihen sind - dann Loop durch Element
    # wenn nicht, dann check ob gesamte Issue-Angabe Heftnrreihe ist, dann Loop
    issuesIt = []
    for issueStr in issues:
        issueComp = re.findall("–|–",result["issue"]["value"])
        if issueComp:
            issuesStr = re.split("–",issueStr)
            #Hier zuerst issueStr splitten, dann loopen
            for i in range(int(issuesStr[0]))
            issuesIt.append
        
    if(len(issuesIt)==0): 
        x = re.findall("–|–",result["issue"]["value"])
        if x:
            for i in range(int(issues[0]),int(issues[1])+1):
                #print(i)
                issuesIt.append(i)
    
    issues = issuesIt
    print(issues)
            
    pageRanges = re.split(";|,",result["pages"]["value"]) #means a range
    pageRanges = list(filter(None,pageRanges)) #entfernt leere Listeneinträge
    print (pageRanges)
    try:
        if (len(issues)==len(pageRanges)):
            print(result["Die_Gartenlaube"]["value"]+"\tArrays are ident")
            #print(issues[0]+" bis "+issues[1])
            j = 0
            for i in range(int(issues[0]), int(issues[1])+1):
                #print(i)
                #print(pageRanges[j])
                j = j+1
        else:
            print(result["Die_Gartenlaube"]["value"]+"\tarrays are not similar")
            
        
    except:
        print("not a range")
    print("== Next Item ===================================================================================")


31–32
488–490;505–508
[31, 32]
['488–490', '505–508']
http://www.wikidata.org/entity/Q15976449	Arrays are ident
== Next Item ===================================================================================
14–17
193–196, 209-212, 225-228, 237-242
[14, 15, 16, 17]
['193–196', ' 209-212', ' 225-228', ' 237-242']
http://www.wikidata.org/entity/Q19150664	Arrays are ident
== Next Item ===================================================================================
7–9
85–88, 101–104, 113–118
[7, 8, 9]
['85–88', ' 101–104', ' 113–118']
http://www.wikidata.org/entity/Q19151051	Arrays are ident
== Next Item ===================================================================================
2–4
17–20;46–48;49-53;
[2, 3, 4]
['17–20', '46–48', '49-53']
http://www.wikidata.org/entity/Q19151808	Arrays are ident
== Next Item ===================================================================================
33; 37
384-386; 437-438
[2, 3, 4]
['384-386', ' 437-438']
http://www.wikidata.org/entit